In [ ]:
%pip install pandas numpy pathlib openpyxl pyarrow fastparquet
# necessária para poder gerar o arquivo em xlsx


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import openpyxl
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.orc as orc

In [ ]:
# identificando o diretorio onde os dados serão gravados
output_dir = Path.cwd() / "dados"
output_dir.mkdir(exist_ok=True)

print(f"Gerando arquivos em: {output_dir.resolve()}\n")

In [4]:
# 1) Configuração de Produtos e seus respectivos intervalos de preços
produtos_precos = {
    'Notebook': (2500.0, 7000.0),
    'Smartphone': (1200.0, 5000.0),
    'Monitor': (600.0, 2000.0),
    'Teclado Gamer': (150.0, 600.0),
    'Mouse Wireless': (50.0, 300.0),
    'Cadeira Gamer': (800.0, 2500.0)
}

n_registros = 100
lista_produtos = np.random.choice(list(produtos_precos.keys()), n_registros)

In [5]:
# 2️) Gerando valores unitários coerentes com o produto
valores_unitarios = [
    round(np.random.uniform(produtos_precos[p][0], produtos_precos[p][1]), 2)
    for p in lista_produtos
]

# Função para formatar moeda padrão BR
def formatar_moeda_br(valor):
    return f"{valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

In [6]:
# 3️) Gerando estrutura com formatação brasileira
data = {
    'id_transacao': list(range(1, n_registros + 1)),
    
    # Data no formato dd/mm/yyyy
    'data': [d.strftime('%d/%m/%Y') 
             for d in pd.date_range(start='2024-01-01', periods=n_registros, freq='D')],
    
    'produto': lista_produtos.tolist(),
    
    'quantidade': np.random.randint(1, 6, n_registros).tolist(),
    
    # Valor unitário formatado como string monetária BR
    'valor_unitario': [formatar_moeda_br(v) for v in valores_unitarios],
    
    'cliente_regiao': np.random.choice(
        ['Norte', 'Sul', 'Leste', 'Oeste', 'Centro'], 
        n_registros
    ).tolist(),
    
    'status': np.random.choice(
        ['Aprovado', 'Pendente', 'Cancelado'], 
        n_registros, 
        p=[0.7, 0.2, 0.1]
    ).tolist()
}

df = pd.DataFrame(data)

In [ ]:
# Calculando valor total já formatado (forçando problema típico de limpeza)
df['valor_total'] = [
    formatar_moeda_br(q * v)
    for q, v in zip(
        df['quantidade'], 
        valores_unitarios
    )
]

print(f"Massa de dados com {len(df)} registros gerada com sucesso!")
df.head()

In [24]:
 # 1. CSV (Encoding ISO-8859-1 para testar erro de acentuação)
df.to_csv(output_dir / "dados_vendas.csv", index=False, sep=';', encoding='ISO-8859-1')
print("- dados_vendas.csv (OK)")

- dados_vendas.csv (OK)


In [25]:
# 2. TXT (Separado por Tabulação)
df.to_csv(output_dir / "dados_vendas.txt", index=False, sep=';', encoding='utf-8')
print("- dados_vendas.txt (OK)")

- dados_vendas.txt (OK)


In [10]:
# 3. JSON (Formato de lista de registros)
df.to_json(output_dir / "dados_vendas.json", orient='records', indent=4)
print("- dados_vendas.json (OK)")

- dados_vendas.json (OK)


In [18]:
# 4. EXCEL (XLSX)
df.to_excel(output_dir / "dados_vendas.xlsx", index=False)
print("- dados_vendas.xlsx (OK)")

- dados_vendas.xlsx (OK)


In [21]:
# Escrita do Parquet e ORC sem metadados do Pandas
for col in ['data', 'produto', 'cliente_regiao', 'status']:
    df[col] = df[col].astype(str)

try:
    # O segredo: storage_options={"schema": ...} ou converter para table primeiro
    # Mas o jeito mais simples e eficaz de "limpar" o conflito é:


    # Convertendo o DataFrame em uma Tabela PyArrow pura (isso limpa os metadados do Pandas)
    table = pa.Table.from_pandas(df, preserve_index=False)
    
    # Salvando a tabela diretamente pelo motor do PyArrow
    pq.write_table(table, output_dir / "dados_vendas.parquet")
    
    print("- dados_vendas.parquet (OK)")

except Exception as e:
    print(f"! Erro no Parquet: {e}")

- dados_vendas.parquet (OK)


In [23]:
# Escrita do ORC sem metadados conflitantes do Pandas
try:
    # 1. Converte o DataFrame para uma Tabela PyArrow pura
    # Isso remove os metadados 'pandas.period' que geram o erro de extensão
    table_orc = pa.Table.from_pandas(df, preserve_index=False)
    
    # 2. Grava o arquivo ORC usando o driver nativo do PyArrow
    orc.write_table(table_orc, output_dir / "dados_vendas.orc")
    
    print("- dados_vendas.orc (OK)")

except Exception as e:
    print(f"! Erro no ORC: {e}")

- dados_vendas.orc (OK)
